In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import pymongo
from pymongo import MongoClient

In [2]:
# Initialize Spark context
sc = pyspark.SparkContext(appName="Pi")
spark = SparkSession(sc)

24/06/22 18:59:25 WARN Utils: Your hostname, dsbda-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/22 18:59:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/22 18:59:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read the CSV file
csv_file = "hdfs://localhost:54310/finalDSBD/output/part-00000"
df = spark.read.csv(csv_file, inferSchema=True, header=False)

In [4]:
df.show()

+---+-------+--------+-----------+---+---+---------+---+---+----+----+----+----+-----------+----+------+-----+-----+------+----+
|_c0|    _c1|     _c2|        _c3|_c4|_c5|      _c6|_c7|_c8| _c9|_c10|_c11|_c12|       _c13|_c14|  _c15| _c16| _c17|  _c18|_c19|
+---+-------+--------+-----------+---+---+---------+---+---+----+----+----+----+-----------+----+------+-----+-----+------+----+
|  0|student|  single|   basic.4y| no|yes| cellular|apr|wed| 371|   2| 999|   0|nonexistent|-1.8|93.075|-47.1|1.405|5099.1| 1.0|
|  0|student|  single|   basic.4y|yes| no| cellular|sep|wed| 396|   2|   6|   3|    success|-1.1|94.199|-37.5|0.876|4963.6| 1.0|
|  0|student|  single|   basic.6y|yes| no| cellular|sep|thu| 161|   2|   6|   4|    success|-1.1|94.199|-37.5|0.879|4963.6| 1.0|
|  0|student|  single|   basic.9y| no|yes|telephone|jul|mon| 567|   1|   6|   2|    success|-1.7|94.215|-40.3|0.827|4991.6| 1.0|
|  0|student|  single|   basic.9y|yes| no| cellular|aug|tue| 642|   1| 999|   0|nonexistent|-2.9|

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)



Name the columns as they were

In [6]:
columns = ["age", "job", "marital", "education", "housing",
           "loan", "contact", "month", "day_of_week", 
           "duration", "campaign", "pdays", "previous", "poutcome", 
           "emp_var_rate", "cons_price_idx", "cons_conf_idx", 
           "euribor3m", "nr_employed", "y"]

In [7]:
from pyspark.sql.functions import when

# Assign column names
df = df.toDF(*columns)

In [8]:
df.show()

+---+-------+--------+-----------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|    job| marital|  education|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+-------+--------+-----------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|  0|student|  single|   basic.4y|     no| yes| cellular|  apr|        wed|     371|       2|  999|       0|nonexistent|        -1.8|        93.075|        -47.1|    1.405|     5099.1|1.0|
|  0|student|  single|   basic.4y|    yes|  no| cellular|  sep|        wed|     396|       2|    6|       3|    success|        -1.1|        94.199|        -37.5|    0.876|     4963.6|1.0|
|  0|student|  single|   basic.6y|    yes|  no| cellula

In [9]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp_var_rate: double (nullable = true)
 |-- cons_price_idx: double (nullable = true)
 |-- cons_conf_idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr_employed: double (nullable = true)
 |-- y: double (nullable = true)



In [10]:
#changing y from float to int
from pyspark.sql.functions import col, when
df = df.withColumn("y", when(col("y") == 1.0, 1).otherwise(0).cast("int"))

In [11]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp_var_rate: double (nullable = true)
 |-- cons_price_idx: double (nullable = true)
 |-- cons_conf_idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr_employed: double (nullable = true)
 |-- y: integer (nullable = false)



In [12]:
# Convert the Spark DataFrame to a Pandas DataFrame
data = df.toPandas()

In [13]:
data.head()

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,student,single,basic.4y,no,yes,cellular,apr,wed,371,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,1
1,0,student,single,basic.4y,yes,no,cellular,sep,wed,396,2,6,3,success,-1.1,94.199,-37.5,0.876,4963.6,1
2,0,student,single,basic.6y,yes,no,cellular,sep,thu,161,2,6,4,success,-1.1,94.199,-37.5,0.879,4963.6,1
3,0,student,single,basic.9y,no,yes,telephone,jul,mon,567,1,6,2,success,-1.7,94.215,-40.3,0.827,4991.6,1
4,0,student,single,basic.9y,yes,no,cellular,aug,tue,642,1,999,0,nonexistent,-2.9,92.201,-31.4,0.884,5076.2,1


-------------------------------------

MongoDB

In [14]:
# Initialize MongoDB client
conn = MongoClient()

In [15]:
# Create a new database and collection
new_db_name = "new_db"
new_collection_name = "new_collection"
db = conn[new_db_name]
collection = db[new_collection_name]

In [16]:
# Insert the Pandas DataFrame into the new MongoDB collection
# Convert DataFrame to a dictionary and insert it into MongoDB
data_dict = data.to_dict("records")
collection.insert_many(data_dict)

InsertManyResult([ObjectId('6677039317dc4a5cafddadfc'), ObjectId('6677039317dc4a5cafddadfd'), ObjectId('6677039317dc4a5cafddadfe'), ObjectId('6677039317dc4a5cafddadff'), ObjectId('6677039317dc4a5cafddae00'), ObjectId('6677039317dc4a5cafddae01'), ObjectId('6677039317dc4a5cafddae02'), ObjectId('6677039317dc4a5cafddae03'), ObjectId('6677039317dc4a5cafddae04'), ObjectId('6677039317dc4a5cafddae05'), ObjectId('6677039317dc4a5cafddae06'), ObjectId('6677039317dc4a5cafddae07'), ObjectId('6677039317dc4a5cafddae08'), ObjectId('6677039317dc4a5cafddae09'), ObjectId('6677039317dc4a5cafddae0a'), ObjectId('6677039317dc4a5cafddae0b'), ObjectId('6677039317dc4a5cafddae0c'), ObjectId('6677039317dc4a5cafddae0d'), ObjectId('6677039317dc4a5cafddae0e'), ObjectId('6677039317dc4a5cafddae0f'), ObjectId('6677039317dc4a5cafddae10'), ObjectId('6677039317dc4a5cafddae11'), ObjectId('6677039317dc4a5cafddae12'), ObjectId('6677039317dc4a5cafddae13'), ObjectId('6677039317dc4a5cafddae14'), ObjectId('6677039317dc4a5cafddae

In [17]:
# Verify the insertion by printing the collection names in the new database
print(f"Collections in database '{new_db_name}':", db.list_collection_names())

Collections in database 'new_db': ['new_collection']


In [18]:
cur = collection.find({"marital":"single"} ) #Just checking if eveything is okay
for r in cur:
    print(r)

{'_id': ObjectId('6677039317dc4a5cafddadfc'), 'age': 0, 'job': 'student', 'marital': 'single', 'education': 'basic.4y', 'housing': 'no', 'loan': 'yes', 'contact': 'cellular', 'month': 'apr', 'day_of_week': 'wed', 'duration': 371, 'campaign': 2, 'pdays': 999, 'previous': 0, 'poutcome': 'nonexistent', 'emp_var_rate': -1.8, 'cons_price_idx': 93.075, 'cons_conf_idx': -47.1, 'euribor3m': 1.405, 'nr_employed': 5099.1, 'y': 1}
{'_id': ObjectId('6677039317dc4a5cafddadfd'), 'age': 0, 'job': 'student', 'marital': 'single', 'education': 'basic.4y', 'housing': 'yes', 'loan': 'no', 'contact': 'cellular', 'month': 'sep', 'day_of_week': 'wed', 'duration': 396, 'campaign': 2, 'pdays': 6, 'previous': 3, 'poutcome': 'success', 'emp_var_rate': -1.1, 'cons_price_idx': 94.199, 'cons_conf_idx': -37.5, 'euribor3m': 0.876, 'nr_employed': 4963.6, 'y': 1}
{'_id': ObjectId('6677039317dc4a5cafddadfe'), 'age': 0, 'job': 'student', 'marital': 'single', 'education': 'basic.6y', 'housing': 'yes', 'loan': 'no', 'conta

In [19]:
sc.stop()